In [448]:
from openpyxl import load_workbook, formatting, styles
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font
import pandas as pd
import re
import xlrd

In [349]:
# #從第一份文件找出要的GPIO
# def step1_sortGPIO(filePath, sheetName, GPIORow):
#     data = pd.read_excel(filePath2, sheet_name = sheetName).iloc[:,GPIORow]#看GPIO在哪一行 可隨時修改
# #     print(data)
#     GPIOList0 = data.tolist()
# #     GPIOList0 = [ GPIO.replace(' ', "") for GPIO in  GPIOList0 if type(GPIO) == str  else GPIO]
#     GPIOList1 = []
#     for g in GPIOList0:
#         GPIOList1.append(g.replace("", "")) if type(g) == str else GPIOList1.append(g)#去掉GPIO的空格
                
#     return GPIOList1

#從第一份文件找出要的GPIO
def step11_sortGPIO(filePath, sheetName, GPIOCol):
    wb = load_workbook(filePath)
    ws = wb[sheetName]
    GPIOList0 = []
    for i in ws[GPIOCol]:
        GPIOList0.append(i.value)
    return GPIOList0

#從第二份文件找出要的GPIO 的 ball name
def step2_sortBallName(filePath, sheetName):

    data = pd.read_excel(filePath, sheet_name = sheetName).iloc[:,:2]

    GPIOList2 = []
    for idx in data.index:

        eachGPIO ={"GPIO": data.iloc[idx,1].split('/')[0].replace(" ", "") ,"ball" : data.iloc[idx,0]}
        GPIOList2.append(eachGPIO)
    return GPIOList2

#把第二份的ballname map過去第一份並產生dictionary
def step3_map_GPIO_ballName(GPIOList1, GPIOList2):
    ballNameCol = []
    for GPIO in GPIOList1:
        data ={"GPIO" : GPIO, "BallName" : ""}
        if type(GPIO) == str: #如果有list1有GPIO不是nan
            for i in GPIOList2:#跟GPIO
                if GPIO == i["GPIO"]:
                    data["BallName"] = i["ball"]
            ballNameCol.append(data)
        else:        
            ballNameCol.append(data)        
    return ballNameCol

#把整理好的GPIO 跟 ball name 放進Intel file李
def step4_insert_GPIO_ballName(mainFilePath, sheetName, ballNameCol):
    wb = load_workbook(mainFilePath)
    ws = wb[sheetName]
    ws.insert_cols(3,2) #從第三航插入兩列
    
    #開始insert
    num = 0
    
    #styles
    textFont = Font(name='Verdana', size=10)
    textAlignment = Alignment(vertical='center')
    
    titlefont = Font(name='Verdana', size=10, color = "FF00B050", family=2.0, bold=True)
    titlefontAlignment = Alignment(horizontal='center',vertical='center')
    
    
    for i in ws['C']:
        i.value = ballNameCol[num]['GPIO']
        i.font = textFont
        i.alignment = textAlignment
        
        ws.cell(row= num +1 , column= 4).value = ballNameCol[num]['BallName'] #保持num和i是同一列
        ws.cell(row= num +1 , column= 4).font = textFont
        ws.cell(row= num +1 , column= 4).alignment = textAlignment
        
        num += 1
        
    ws['C2'] = "GPIO\n(reference)"
    ws['D2'] = "Ball Name"
    ws['D2'].font = Tfont
    ws['D2'].alignment = TAlignment
    ws['C2'].font = Tfont
    ws['C2'].alignment = TAlignment
    
    wb.save(f"{mainFilePath}_+ballName.xlsx")

In [262]:
#original step1

# #step1
# #先找出要的GPIO
# filePath, sheetName, GPIORow = 'INTEL materials/627075_ADL_P_PCH_GPIO_IS_Rev1p0.xlsx', 'GPIO Implementation', 1
# try:
#     GPIOList1 = step1_sortGPIO(filePath, sheetName, GPIORow)
#     print("step 1: Extract GPIO List Success.")
# except:
#     print("Clear Data Fail, check step 1: 'function step1_sortGPIO' ")

step 1: Extract GPIO List Success.


In [463]:
#step1
#先找出要的GPIO
filePath, sheetName, GPIOCol = 'INTEL materials/627075_ADL_P_PCH_GPIO_IS_Rev1p0.xlsx', 'GPIO Implementation', "B"
try:
    GPIOList1 = step11_sortGPIO(filePath, sheetName, GPIOCol)
    print("step 1: Extract GPIO List Success.")
except:
    print("Clear Data Fail, check step 1: 'function step11_sortGPIO' ")
    
#step2
filePath, sheetName = 'INTEL materials/intel_template_1.xlsm', 'Pinlist'
try:
    GPIOList2 = step2_sortBallName(filePath, sheetName)
    print("step 2: Extract CPU Ball Name Success.")
except:
    print("Clear Data Fail, check step 2: 'step2_sortBallName' ")

    
#step3
try:
    ballNameCol = step3_map_GPIO_ballName(GPIOList1, GPIOList2)
    print("step 3: Mapping Ballname Success.")
except:
    print("Mapping Ballname Fail, check step 3: 'step3_map_GPIO_ballName' ")
    
#step4 insert ballname

mainFilePath = 'INTEL materials/627075_ADL_P_PCH_GPIO_IS_Rev1p0.xlsx'
sheetName = 'GPIO Implementation'

try:
    step4_insert_GPIO_ballName(mainFilePath, sheetName, ballNameCol)
    print("step 4: Insert Ballname Success.")
except:
    print("Inserting Ballname Fail, check step 4: 'step4_insert_GPIO_ballName' ")

step 1: Extract GPIO List Success.
step 2: Extract CPU Ball Name Success.
step 3: Mapping Ballname Success.


In [351]:
#chekcPart
print(len(GPIOList1))
print(len(GPIOList2))
print(len(ballNameCol))
ballNameCol

196
1751
196


[{'GPIO': None, 'BallName': ''},
 {'GPIO': 'GPIO', 'BallName': ''},
 {'GPIO': None, 'BallName': ''},
 {'GPIO': 'GPP_A0', 'BallName': 'DP51'},
 {'GPIO': 'GPP_A1', 'BallName': 'DT44'},
 {'GPIO': 'GPP_A2', 'BallName': 'DT54'},
 {'GPIO': 'GPP_A3', 'BallName': 'DP52'},
 {'GPIO': 'GPP_A4', 'BallName': 'DP44'},
 {'GPIO': 'GPP_A5', 'BallName': 'DP47'},
 {'GPIO': 'GPP_A6', 'BallName': 'DP54'},
 {'GPIO': 'GPP_A7', 'BallName': 'EB52'},
 {'GPIO': 'GPP_A8', 'BallName': 'EB54'},
 {'GPIO': 'GPP_A9', 'BallName': 'DT49'},
 {'GPIO': 'GPP_A10', 'BallName': 'DT51'},
 {'GPIO': 'GPP_A11', 'BallName': 'DV51'},
 {'GPIO': 'GPP_A12', 'BallName': 'DY46'},
 {'GPIO': 'GPP_A13', 'BallName': 'DV47'},
 {'GPIO': 'GPP_A14', 'BallName': 'DY47'},
 {'GPIO': 'GPP_A15', 'BallName': 'DY49'},
 {'GPIO': 'GPP_A16', 'BallName': 'DY51'},
 {'GPIO': 'GPP_A17', 'BallName': 'DY54'},
 {'GPIO': 'GPP_A18', 'BallName': 'EB47'},
 {'GPIO': 'GPP_A19', 'BallName': 'EB49'},
 {'GPIO': 'GPP_A20', 'BallName': 'EB51'},
 {'GPIO': 'GPP_A21', 'BallN

In [327]:
ballNameCol
dfballNameCol = pd.DataFrame(ballNameCol)
dfballNameCol

,GPIO,BallName
0,None,NaN
1,GPIO,NaN
2,None,NaN
3,GPP_A0,DP51
4,GPP_A1,DT44
...,...,...
191,None,None
192,None,None
193,None,None
194,None,None


In [499]:
wb = load_workbook('INTEL materials/text.xlsx')
ws = wb['GPIO Implementation']
ws.insert_cols(3,2)

In [ ]:
#chekcPart
print(len(GPIOList1))
print(len(GPIOList2))
print(len(ballNameCol))
ballNameCol

step 4: Insert Ballname Success.


In [507]:
mainFilePath = 'INTEL materials/text.xlsx'
sheetName = 'GPIO Implementation'
num = 0

textFont = Font(name='Verdana', size=10)
textAlignment = Alignment(vertical='center')

               
for i in ws['C']:
    i.value = ballNameCol[num]['GPIO']
    i.font = textFont
    i.alignment = textAlignment
    
        
    ws.cell(row= num +1 , column= 4).value = ballNameCol[num]['BallName'] #保持num和i是同一列
    ws.cell(row= num +1 , column= 4).font = textFont
    ws.cell(row= num +1 , column= 4).alignment = textAlignment
    
   
    num += 1


ws['C2'] = "GPIO\n(reference)"
ws['D2'] = "Ball Name"
titlefont = Font(name='Verdana', size=10, color = "FF00B050", family=2.0, bold=True)
titlefontAlignment = Alignment(horizontal='center',vertical='center', wrapText=True)

ws['D2'].font = Tfont
ws['D2'].alignment = TAlignment
ws['C2'].font = Tfont
ws['C2'].alignment = TAlignment


In [490]:
savenum = 0

In [508]:

wb.save(f"test{savenum}.xlsx")
savenum +=1

In [3]:
wb = load_workbook('INTEL materials/627075_ADL_P_PCH_GPIO_IS_Rev1p0.xlsx')
print(wb.sheetnames)

['Legal Disclaimer', 'Revision History', 'GPIO Implementation']


In [15]:
for GPIO in table['b']:
    print(GPIO.value)

None
GPIO
None
GPP_A0
GPP_A1
GPP_A2
GPP_A3
GPP_A4
GPP_A5
GPP_A6
GPP_A7
GPP_A8
GPP_A9
GPP_A10
GPP_A11
GPP_A12
GPP_A13
GPP_A14
GPP_A15
GPP_A16
GPP_A17
GPP_A18
GPP_A19
GPP_A20
GPP_A21
GPP_A22
GPP_A23
None
GPP_B0
GPP_B1
GPP_B2
GPP_B3
GPP_B4
GPP_B5
GPP_B6
GPP_B7
GPP_B8
None
None
GPP_B11
GPP_B12
GPP_B13
GPP_B14
GPP_B15
GPP_B16
GPP_B17
GPP_B18
None
None
None
None
GPP_B23
None
GPP_C0
GPP_C1
GPP_C2
GPP_C3
GPP_C4
GPP_C5
GPP_C6
GPP_C7
None
GPP_D0
GPP_D1
GPP_D2
GPP_D3
GPP_D4
GPP_D5
GPP_D6
GPP_D7
GPP_D8
GPP_D9
GPP_D10
GPP_D11
GPP_D12
GPP_D13
GPP_D14
GPP_D15
GPP_D16
GPP_D17
GPP_D18
GPP_D19
None
GPP_E0
GPP_E1
GPP_E2
GPP_E3
GPP_E4
GPP_E5
GPP_E6
GPP_E7
GPP_E8
GPP_E9
GPP_E10
GPP_E11
GPP_E12
GPP_E13
GPP_E14
GPP_E15
GPP_E16
GPP_E17
GPP_E18
GPP_E19
GPP_E20
GPP_E21
GPP_E22
GPP_E23
None
GPP_F0
GPP_F1
GPP_F2
GPP_F3
GPP_F4
GPP_F5
GPP_F6
GPP_F7
None
GPP_F9
GPP_F10
GPP_F11
GPP_F12
GPP_F13
GPP_F14
GPP_F15
GPP_F16
GPP_F17
GPP_F18
GPP_F19
GPP_F20
GPP_F21
GPP_F22
GPP_F23
None
GPP_H0
GPP_H1
GPP_H2
GPP_H3
GPP_H4
GPP_H

In [ ]:
border = Border(left   = styles.Side(border_style='mediumDashed', color='FF000000'),
                right  = styles.Side(border_style='mediumDashed', color='FF000000'),
                top    = styles.Side(border_style='mediumDashed', color='FF000000'),
                bottom = styles.Side(border_style='mediumDashed', color='FF000000'),
               )
nonBorder = Border(left   = styles.Side(border_style=None, color='FF000000'),
                   right  = styles.Side(border_style=None, color='FF000000'),
                   top    = styles.Side(border_style=None, color='FF000000'),
                   bottom = styles.Side(border_style=None, color='FF000000'),